In [1]:
import pandas as pd
import numpy as np
import os
import librosa
import seaborn as sb
import matplotlib.pyplot as plt
import librosa.display
from IPython.display import Audio
import warnings
warnings.filterwarnings("ignore")

In [2]:
paths = []
labels = []
import os
for dirname, dirs, filename in os.walk(".", topdown=False):
   for name in filename:
      label = name.split('_')[-1].split('.')[0]
      if label in ['angry','disgust','fear','happy','neutral','ps','sad']:
         labels.append(label)
         paths.append(os.path.join(dirname, name))
df = pd.DataFrame()
df['speech'] = paths
df['label'] = labels
df.head()

,speech,label
0,.\sounds\OAF_angry\OAF_back_angry.wav,angry
1,.\sounds\OAF_angry\OAF_bar_angry.wav,angry
2,.\sounds\OAF_angry\OAF_base_angry.wav,angry
3,.\sounds\OAF_angry\OAF_bath_angry.wav,angry
4,.\sounds\OAF_angry\OAF_bean_angry.wav,angry


In [3]:
df.tail()

,speech,label
2795,.\sounds\YAF_sad\YAF_witch_sad.wav,sad
2796,.\sounds\YAF_sad\YAF_yearn_sad.wav,sad
2797,.\sounds\YAF_sad\YAF_yes_sad.wav,sad
2798,.\sounds\YAF_sad\YAF_young_sad.wav,sad
2799,.\sounds\YAF_sad\YAF_youth_sad.wav,sad


In [4]:
def mfcc_extract(filename):
    y,sr = librosa.load(filename,duration=3,offset=0.5)
    mfcc = np.mean(librosa.feature.mfcc(y=y,sr=sr,n_mfcc=40).T,axis=0)
    return mfcc

In [5]:
x_mfcc = df['speech'].apply(lambda x: mfcc_extract(x))

In [6]:
x = [i for i in x_mfcc]
x = np.array(x)
x = np.expand_dims(x,-1)
x.shape

(2800, 40, 1)

In [7]:
from sklearn.preprocessing import OneHotEncoder
enc = OneHotEncoder()
y = enc.fit_transform(df[['label']])

In [8]:
y = y.toarray()

In [9]:
import tensorflow as tf
from tensorflow.python.keras.layers import Dense
from tensorflow.python.keras.layers import LSTM
from tensorflow.python.keras.layers import Dropout
from tensorflow.python.keras.models import Sequential

In [10]:
model = Sequential([
    LSTM(123,return_sequences=False,input_shape=(40,1)),
    Dense(64,activation='relu'),
    Dropout(0.2),
    Dense(32,activation='relu'),
    Dropout(0.2),
    Dense(7,activation='softmax')

])

model.compile(loss='categorical_crossentropy',optimizer = 'adam',metrics=['accuracy'])

In [11]:
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test=train_test_split(x,y,test_size = 0.2,stratify=y,random_state=43)
hist = model.fit(x_train,y_train,validation_split=0.2,epochs=100,batch_size=256,shuffle=True)

Epoch 1/100
7/7 [==============================] - 3s 189ms/step - loss: 1.8916 - accuracy: 0.2489 - val_loss: 1.7710 - val_accuracy: 0.4107
Epoch 2/100
7/7 [==============================] - 1s 127ms/step - loss: 1.6849 - accuracy: 0.4185 - val_loss: 1.5364 - val_accuracy: 0.4933
Epoch 3/100
7/7 [==============================] - 1s 125ms/step - loss: 1.4842 - accuracy: 0.4816 - val_loss: 1.2729 - val_accuracy: 0.5536
Epoch 4/100
7/7 [==============================] - 1s 123ms/step - loss: 1.2143 - accuracy: 0.5552 - val_loss: 1.0058 - val_accuracy: 0.6094
Epoch 5/100
7/7 [==============================] - 1s 116ms/step - loss: 1.0538 - accuracy: 0.5792 - val_loss: 0.8526 - val_accuracy: 0.6607
Epoch 6/100
7/7 [==============================] - 1s 110ms/step - loss: 0.9016 - accuracy: 0.6222 - val_loss: 0.7029 - val_accuracy: 0.7210
Epoch 7/100
7/7 [==============================] - 1s 109ms/step - loss: 0.7710 - accuracy: 0.6869 - val_loss: 0.5898 - val_accuracy: 0.7879
Epoch 8/100
7

In [12]:
def sparser(arr):
    for i in arr:
        a=max(i)
        b = np.where(i==a)[0]
        for j in range(len(i)):
            i[j]=0
        i[b[0]]=1
    return arr
        

In [13]:
def emotion_checker(test):
    
    emots = ['anger','disgust','fear','happiness','neutral','ps','sad']
    test = mfcc_extract(test)
    test = [[i for i in test]]
    test = np.array(test)
    test = np.expand_dims(test,-1)
    res = model.predict(test)
    res = sparser(res)
    for i in res:
        b = np.where(i==1)[0]
        print("emotion is:",emots[b[0]])

In [22]:
emotion_checker('.\sounds\YAF_happy\YAF_back_happy.wav') 

emotion is: happiness
